# Exercise 3.2 Colormaps and colorbars

We already used colormaps and colorbars for the scatterplots, but we will discuss some more details here.


Note that most of what we show here for georeferenced plots also applies for normal colormaps and bars.

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

%matplotlib inline

In [ ]:
import utils

## Load data

### Function to create artificial data:

In [ ]:
# artificial data


def sample_data_3d(nlons, nlats):
    """Returns `lons`, `lats`, and fake `data`

    adapted from:
    http://scitools.org.uk/cartopy/docs/v0.15/examples/axes_grid_basic.html
    """
    
    dlat = 180. / nlats / 2
    dlon = 360. / nlons

    lat = np.linspace(-90 + dlat, 90 - dlat, nlats)   
    lon = np.linspace(0, 360 - dlon, nlons)

    lons, lats = np.meshgrid(np.deg2rad(lon), np.deg2rad(lat))
    wave = 0.75 * (np.sin(2 * lats) ** 8) * np.cos(4 * lons)
    mean = 0.5 * np.cos(2 * lats) * ((np.sin(2 * lats)) ** 2 + 2)
    data = wave + mean
    
    return lon, lat, data

## CMIP 5, relative precipitation change

Create a netCDF with historical, and projected climatlological precipitation, as well as the relative change between them, from all CMIP5 models for RCP8.5 (Taylor et al., 2012).

The data was prepared in [another notebook](../data/prepare_CMIP5_map.ipynb).

In [ ]:
fN = '../data/cmip5_delta_pr_rcp85_map.nc'

# load data, omitting some unecessary variables
pr = xr.open_dataset(fN, drop_variables=['agree_sign', 'pval'])

pr

## viridis, 

The c

In [ ]:
# create sample data
lon, lat, data = sample_data_3d(90, 48)

# ====

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, vmin=-1, vmax=1)

plt.colorbar(h)

ax.set_global()

In [ ]:
# create sample data
lon, lat, data = sample_data_3d(90, 48)

# ====

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, vmin=-1, vmax=1, cmap='RdBu_r')

plt.colorbar(h)

ax.set_global()

In [ ]:
cmap = plt.cm.get_cmap('RdBu_r', 7)

#cmap.set_over('y')
#print(cmap(3))
#print(cmap(4))

print(cmap(1.0))
print(cmap(1))

cmap([0, 1])



print(cmap([0, 0.1]))


cmap.__call__??

In [ ]:
np.nextafter([1, 1.11, 1.25], [1.1])

In [ ]:
cmap = plt.cm.get_cmap('RdBu_r', 7)



ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, vmin=-1, vmax=0.8, cmap=cmap, transform=ccrs.PlateCarree())

plt.colorbar(h, extend='both')

ax.set_global()

In [ ]:
ccrs.Orthographic?

## Placement of colorbar

We already noted that the colorbar can destroy the layout of a plot:

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=45))

ax.coastlines()
h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, transform=ccrs.PlateCarree())


plt.colorbar(h, extend='both', orientation='horizontal')

ax.set_global()

ax.get_aspect()

The issue is that the aspect ratio of a map plot has to be equal, else it would be distorted. Matplotlib then 'shrinks' the axes, but does not shrink the area of the figure. The colorbar is correct when we set the aspect to 'auto', but then of course the map is wrong...

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=45))

ax.coastlines()
h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, transform=ccrs.PlateCarree())


plt.colorbar(h, extend='both', orientation='horizontal')

ax.set_aspect('auto')

ax.set_global()

# Solution \#1 - using axes_grid1


"[axes_grid1](https://matplotlib.org/2.0.2/mpl_toolkits/axes_grid/users/overview.html) is a collection of helper classes to ease displaying (multiple) images with matplotlib. In matplotlib, the axes location (and size) is specified in the normalized figure coordinates, which may not be ideal for displaying images that needs to have a given aspect ratio."

 > However, it is not part of the core matplotlib functionality, and not it's best-documented part

From the axes_grid1 toolkit we need `make_axes_locatable`:

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

from mpl_toolkits import axes_grid1

In [ ]:
# create normal plot
f = plt.figure()

ax = plt.axes(projection=ccrs.Orthographic(central_latitude=45))

ax.coastlines()

h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, transform=ccrs.PlateCarree(), 
                  vmin=-1, vmax=1, cmap='RdBu')


ax.set_global()

# ===========================

# create axes that has the right size
divider = make_axes_locatable(ax)
cbax = divider.append_axes('bottom', size="6.5%", pad=0.1, axes_class=plt.Axes)

# create colorbar in this axes
cbar = plt.colorbar(h, cax=cbax, orientation='horizontal', extend='both')

cbar.set_ticks(np.arange(-1, 1.1, .5))

Note that you need to pass `axes_class=plt.Axes` to `append_axes`, else it fails miserably (because it tries to create a new axes with a projection). 


### Exercise

 * add a vertical colorbar to the historical precipitation plot (over Europe)

In [ ]:
# create normal plot
f = plt.figure()

ax = plt.axes(projection=ccrs.LambertAzimuthalEqualArea())

ax.coastlines()


h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)


# add colorbar here

If this does not look entirely correct - you may me right - you have to add `ax.set_global()` or `ax.set_extent([...], ccrs.PlateCarree())`!

### Solution

In [ ]:
# create normal plot
f = plt.figure()

ax = plt.axes(projection=ccrs.LambertAzimuthalEqualArea())

ax.coastlines()



h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

# add colorbar here

# this is required
ax.set_global()

# create axes that has the right size
divider = make_axes_locatable(ax)
cbax = divider.append_axes('right', size="6.5%", pad=0.1, axes_class=plt.Axes)

# create colorbar in this axes
cbar = plt.colorbar(h, cax=cbax, orientation='vertical', extend='max')

# Solution \#2 - using `cbax.set_position`

The solution with `axes_grid1` is reasonably that works well, however, it has (afaik) two limitations:
 * the colorbar cannot span more than one axes
 * you cannot shrink the colorbar, e.g. to make room for a label below
 
Therefore I present a second solution. It is inspired by this [stackoverflow answer](https://stackoverflow.com/a/30077745). The trick is to read out the coordinates of the cartopy axes and adjust the position of the colorbar accordingly. Because the position of the cartopy axes can change, we have to redo this everytime the plot get's drawn.



In [ ]:

# function we pass the ax and colorbar axes to
def resize_colorbar(cbax, ax):
    
    # inner function that is called by the event handler
    def inner(event):

        posn = ax.get_position()
        
        left = posn.x0
        bottom = posn.y0 - 0.1
        width = posn.width
        height = 0.05
    
        cbax.set_position([left, bottom, width, height])
        
    return inner


# ===========================

f = plt.figure()
ax = plt.axes(projection=ccrs.Orthographic(central_latitude=45))

ax.coastlines()

h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, transform=ccrs.PlateCarree(), 
                  vmin=-1, vmax=1, cmap='RdBu')

ax.set_global()

# ===================

# we need to add some random axes
cbax = f.add_axes([0, 0, 0.1, 0.1])

# create the colorbar
cbar = plt.colorbar(h, cax=cbax, orientation='horizontal')

# pass cbax and ax to our function
func = resize_colorbar(cbax, ax)

# create an event handler
# every time an draw event is observed `func(event)` is called

f.canvas.mpl_connect('draw_event', func)

# tell mpl to draw
plt.draw()


### Bonus exercise

 * rewrite the function for a vertical colorbar
 > hint: posn contains: `posn.x0`, `posn.y0`, `posn.width`, `posn.height`
 * align the top with the top of the plot, but shrink it a bit
 * add the units (`'[mm]'`) below the colorbar (use cbax.set_xlabel)

In [ ]:
# change the calculation of left, bottom, width, and height

def resize_colorbar(cbax, ax):
    
    # inner function that is called by the event handler
    def inner(event):

        posn = ax.get_position()
        
        left = posn.x0
        bottom = posn.y0 - 0.1
        width = posn.width
        height = 0.05
    
        cbax.set_position([left, bottom, width, height])
        
    return inner


# ===========================


# create normal plot
f = plt.figure()

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()



h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

# add colorbar here

# this is required
ax.set_global()




# we need to add some random axes
cbax = f.add_axes([0, 0, 0.1, 0.1])

# create the colorbar
cbar = plt.colorbar(h, cax=cbax, orientation='vertical')

# pass cbax and ax to our function
func = resize_colorbar(cbax, ax)


f.canvas.mpl_connect('draw_event', func)

# tell mpl to draw
plt.draw()

# add the units



### Solution

In [ ]:
# change the calculation of left, bottom, width, and height

def resize_colorbar(cbax, ax):
    
    # inner function that is called by the event handler
    def inner(event):

        posn = ax.get_position()
        
        left = posn.x0 + posn.width + 0.03
        bottom = posn.y0 + 0.08
        width = 0.025
        height = posn.height - 0.08
    
        cbax.set_position([left, bottom, width, height])
        
    return inner


# ===========================


# create normal plot
f = plt.figure()

ax = plt.axes(projection=ccrs.Robinson())

ax.coastlines()



h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

# add colorbar here

# this is required
ax.set_global()




# we need to add some random axes
cbax = f.add_axes([0, 0, 0.1, 0.1])

# create the colorbar
cbar = plt.colorbar(h, cax=cbax, orientation='vertical')

# pass cbax and ax to our function
func = resize_colorbar(cbax, ax)


f.canvas.mpl_connect('draw_event', func)

# tell mpl to draw
plt.draw()

# add the units

cbax.set_xlabel('[mm]', labelpad=10);

This solution requires a bit more code (and thinking), but we can do whatever we want, also expand it to more than one axes... Before you go wild coding - I did that for you. `utils` contains the fuctions `resize_colorbar_horz` and `resize_colorbar_vert`:

In [ ]:
f = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

h = ax.pcolormesh(*utils.infer_interval_breaks(lon, lat), data, transform=ccrs.PlateCarree(), 
                  vmin=-1, vmax=1, cmap='RdBu_r')

ax.set_global()

# ===================

# we need to add some random axes
cbax = f.add_axes([0, 0, 0.1, 0.1])

# create the colorbar
cbar = plt.colorbar(h, cax=cbax, orientation='vertical')

func = utils.resize_colorbar_vert(cbax, ax, pad=0.025, size=0.03)

f.canvas.mpl_connect('draw_event', func)

plt.draw()

### Exercise
 * use `utils.resize_colorbar_horz` to make a colorbar that spans both axes in the plot showing historical and projected rainfall amounts
 * save the figure as a pdf
 * is the colorbar at the right position in the saved figure?

In [ ]:
utils.resize_colorbar_horz?

In [ ]:
# create normal plot
f, axes = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = axes[0]
h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

ax.set_title('Precipication: historical')

ax = axes[1]
h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.proj, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

ax.set_title('Precipication: projections')


for ax in axes:
    ax.coastlines()
    ax.set_global()



# we need to add some random axes
cbax = f.add_axes([0, 0, 0.1, 0.1])

# create the colorbar
cbar = plt.colorbar(h, cax=cbax, orientation='horizontal', extend='max')



### Solution

If the colorbar is not at the right position in the saved figure, you forgot plt.draw()!

In [ ]:
# create normal plot
f, axes = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = axes[0]
h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

ax.set_title('Precipication: historical')

ax = axes[1]
h = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.proj, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

ax.set_title('Precipication: projections')


for ax in axes:
    ax.coastlines()
    ax.set_global()



# we need to add some random axes
cbax = f.add_axes([0, 0, 0.1, 0.1])

# create the colorbar
cbar = plt.colorbar(h, cax=cbax, orientation='horizontal', extend='max')

# pass cbax and ax to our function
func = utils.resize_colorbar_horz(cbax, axes[0], axes[1])

f.canvas.mpl_connect('draw_event', func)

# tell mpl to draw
plt.draw()

# add the units

cbar.set_label('[mm]')

# plt.savefig('pr_hist_proj.pdf')

### Exercise

Here we plot historical precipitation and the relative change to the projection.
 
 * Create the second horizontal colorbar and use `resize_colorbar_horz`
 > don't forget to adjust the label, the ticks and replace h0 with h1

In [ ]:
# create normal plot
f, axes = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = axes[0]
h0 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

ax.set_title('Precipication: historical')

ax = axes[1]
h1 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.pr_rel, transform=ccrs.PlateCarree(), 
                  cmap='BrBG', vmin=-50, vmax=50)

ax.set_title('Precipication: projections')


for ax in axes:
    ax.coastlines()
    ax.set_global()



# create the first colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h0, cax=cbax, orientation='horizontal', extend='max')
func = utils.resize_colorbar_horz(cbax, axes[0])
f.canvas.mpl_connect('draw_event', func)
cbar.set_label('[mm]')
cbar.set_ticks(np.arange(0, 4100, 1000))

# create the second colorbar

plt.draw()


### Why does this not work? (If it does, congratulations!)

You probably copied the code, especially the line 

    cbax = f.add_axes([0, 0, 0.1, 0.1])

If you create axes at the *same position* twice it is the same axes! Thus you need to adjust the location of the axes slightly.

### Solution

In [ ]:
# create normal plot
f, axes = plt.subplots(1, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))

ax = axes[0]
h0 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)

ax.set_title('Precipication: historical')

ax = axes[1]
h1 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.pr_rel, transform=ccrs.PlateCarree(), 
                  cmap='BrBG', vmin=-50, vmax=50)

ax.set_title('Precipication: change')


for ax in axes:
    ax.coastlines()
    ax.set_global()



# create the first colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h0, cax=cbax, orientation='horizontal', extend='max')
func = utils.resize_colorbar_horz(cbax, axes[0])
f.canvas.mpl_connect('draw_event', func)
cbar.set_label('[mm]')
cbar.set_ticks(np.arange(0, 4100, 1000))

# create the second colorbar

# note the '0.11'
cbax = f.add_axes([0, 0, 0.1, 0.11])
cbar = plt.colorbar(h1, cax=cbax, orientation='horizontal', extend='both')
func = utils.resize_colorbar_horz(cbax, axes[1])
f.canvas.mpl_connect('draw_event', func)
cbar.set_label('[%]')
cbar.set_ticks(np.arange(-50, 51, 20))

plt.draw()

## Some more stuff you can do:

In [ ]:
f, axes = plt.subplots(2, 3, subplot_kw=dict(projection=ccrs.PlateCarree()))

axes = axes.flatten()

for ax in axes:
    ax.coastlines() 
    ax.set_global()

h0 = ax.pcolormesh([[0, 1]])
h1 = ax.pcolormesh([[0, 1]])
h2_1 = ax.pcolormesh([[0, 1]], cmap='Blues')

h2_2 = ax.pcolormesh([[0, 1]], cmap='Reds')


h3 = ax.pcolormesh([[0, 1]], cmap='BrBG')

cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h1, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[0], axes[1], size=0.02, pad=0.025)
f.canvas.mpl_connect('draw_event', func)

cbax = f.add_axes([0, 0, 0.1, 0.11])
cbar = plt.colorbar(h2_1, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[2], size=0.02, pad=0.025)
f.canvas.mpl_connect('draw_event', func)
cbax.set_xticklabels([])

cbax = f.add_axes([0, 0, 0.1, 0.12])
cbar = plt.colorbar(h2_2, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[2], size=0.02, pad=0.075)
f.canvas.mpl_connect('draw_event', func)


cbax = f.add_axes([0, 0, 0.1, 0.13])
cbar = plt.colorbar(h3, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[3], axes[-1], size=0.02, pad=0.025)
f.canvas.mpl_connect('draw_event', func)

plt.draw()

## shrink and shift

you can use the `shrink` and `shift` keywords to adjust the position of the colorbar. `shrink` and `shift` are in fraction of the total width/ height of the colorbar.


In [ ]:
f, axes = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()))

axes = axes.flatten()

for ax in axes:
    ax.set_global()

h = ax.pcolormesh([[0, 1]])

cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[0], size=0.02, pad=0.05, shrink=0.5)
f.canvas.mpl_connect('draw_event', func)

axes[0].text(0.5, 0.5, "shrink=0.5\nshift='symmetric' (default)", ha='center', va='center')


cbax = f.add_axes([0, 0, 0.1, 0.11])
cbar = plt.colorbar(h, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[1], size=0.02, pad=0.05, shrink=0.5, shift=0.35)
f.canvas.mpl_connect('draw_event', func)

axes[1].text(0.5, 0.5, 'shrink=0.5\nshift=0.35', ha='center', va='center')

cbax = f.add_axes([0, 0, 0.1, 0.12])
cbar = plt.colorbar(h, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[2], size=0.02, pad=0.05, shrink=0.5, shift=0.0)
f.canvas.mpl_connect('draw_event', func)

axes[2].text(0.5, 0.5, 'shrink=0.5\nshift=0.', ha='center', va='center')

cbax = f.add_axes([0, 0, 0.1, 0.13])
cbar = plt.colorbar(h, orientation='horizontal', cax=cbax)
func = utils.resize_colorbar_horz(cbax, axes[3], size=0.02, pad=0.05, shift=0.2)
f.canvas.mpl_connect('draw_event', func)

axes[3].text(0.5, 0.5, 'shrink=None (default)\nshift=0.2', ha='center', va='center')

plt.draw()

### Exercise
 * add one colorbar for the climatologies and one for the relative change
 * add the units with `cbax.set_xlabel`
 * use `shrink` and `shift` to make room for the xlabel

In [ ]:
# create normal plot
f, axes = plt.subplots(3, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))

f.set_size_inches(w=10 / 2.54, h=16 / 2.54)


ax = axes[0]
h0 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)


ax = axes[1]
h1 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.proj, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)


ax = axes[2]
h2 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.pr_rel, transform=ccrs.PlateCarree(), 
                  cmap='BrBG', vmin=-50, vmax=50)



axes[0].set_title('Precipication: historical')
axes[1].set_title('Precipication: projections')
axes[2].set_title('Precipication: change')

for ax in axes:
    ax.coastlines()
    ax.set_global()


# first colorbar


# second colorbar


plt.draw()

### Solution

As shift is relative, I use `shift=0.1` for the upper colorbar and `shift=0.2` for the lower colorbar. This creates approximately the same absolute shift.

In [ ]:
# create normal plot
f, axes = plt.subplots(3, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))

f.set_size_inches(w=10 / 2.54, h=16 / 2.54)


ax = axes[0]
h0 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.hist, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)


ax = axes[1]
h1 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.proj, transform=ccrs.PlateCarree(), 
                  cmap='Blues', vmin=0, vmax=4000)


ax = axes[2]
h2 = ax.pcolormesh(*utils.infer_interval_breaks(pr.lon, pr.lat), pr.pr_rel, transform=ccrs.PlateCarree(), 
                  cmap='BrBG', vmin=-50, vmax=50)



axes[0].set_title('Precipication: historical')
axes[1].set_title('Precipication: projections')
axes[2].set_title('Precipication: change')

for ax in axes:
    ax.coastlines()
    ax.set_global()


# first colorbar
cbax = f.add_axes([0, 0, 0.1, 0.1])
cbar = plt.colorbar(h1, cax=cbax, orientation='vertical', extend='max')
func = utils.resize_colorbar_vert(cbax, axes[0], axes[1], shift=0.1)
f.canvas.mpl_connect('draw_event', func)
cbax.set_xlabel('[mm]')



# second colorbar
cbax = f.add_axes([0, 0, 0.1, 0.11])
cbar = plt.colorbar(h2, cax=cbax, orientation='vertical', extend='both')
func = utils.resize_colorbar_vert(cbax, axes[2], shift=0.2)
f.canvas.mpl_connect('draw_event', func)
cbax.set_xlabel('[%]')



plt.draw()